# PP6 neural networks for tabular data

<a href="https://colab.research.google.com/github/PauliusU/PP6-neural-networks-for-tabular-data/blob/master/PP6-neural-networks-for-tabular-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PauliusU/PP6-neural-networks-for-tabular-data/blob/master/PP6-neural-networks-for-tabular-data.ipynb)